In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import linalg, diags
from scipy import integrate
from scipy.stats import norm

# list of parameters
dt = 0.01
dS = 1
S_low, S_high = 50, 55
step_no = int((S_high - S_low)/dS -1)
time_no = int((end_time - start_time)/dt + 1)
start_time, end_time = 0,1
strike_price = 52


def int_rate(t):
    return 0.05

def div_rate(t):
    return 0.01

def volatility(asset, t):
    return 0.2

X,Y = np.meshgrid(np.arange(S_low+dS, S_high, dS), np.arange(start_time, end_time+dt, dt))

def integ(func, arr): #expected arr entries are of the form: np.arange(start_time, end_time, timestep)
    arr = np.flip(arr, 0) # flip to count form back to front
    out_list = []
    for i in range(1,len(arr)):
        increment = arr[i-1] - arr[i]
        to_add = increment * func((arr[i-1]+ arr[i])/2)
        out_list.append(to_add)
    out_arr = np.array([0] + out_list)
    return np.cumsum(out_arr)[::-1]

def payoff(asset):
    return np.clip(asset-strike_price, 0, None)

integ(lambda x: x**2, np.arange(0,5,1))
# norm.cdf(np.arange(1, 0, -0.1))- integ(norm.ppf, np.arange(0,1,0.1))

In [ ]:
def second_order_coeff(asset, t):
    return (asset * volatility(asset, t)) ** 2/2

def first_order_coeff(asset, t):
    return (int_rate(t) - div_rate(t)) * asset

def zero_order_coeff(asset, t):
    return -int_rate(t)

v1 = dt/dS
v2 = dt/dS

a, b, c = second_order_coeff(X,Y), first_order_coeff(X, Y), zero_order_coeff(X, Y)
A = v1 * a/2  - v2 * b/4
B = -v1 *a + dt * c/2
C = v1 * a/2 + v2 * b/4   # all these start from S_1, 0 and end at S_{M-1} , T
A_1, C_M = A[:,0], C[:,-1]
lower_bdd, upper_bdd = strike_price * np.exp(- integ(int_rate, np.arange(start_time, end_time+dt, dt))), np.zeros(time_no)
# lower_bdd and Upper_bdd counted FROM BACKWARDS


In [ ]:
def boundary_addon(t, case = 'Dirichlet'):
    # just for call-option
    time_idx = ((t - start_time)/dt).astype(int)
    if case == 'Dirichlet':
        
        
        
        
#         lower_bdd = np.zeros(len(time_set))
#         upper_bdd = S_high * np.exp(-integ(div_rate, time_set)) \
#             - strike_price * np.exp (- integ(int_rate, time_set))
#     else:
#         lower_bdd = -S_low * np.exp(-integ(div_rate, time_set)) \
#             + strike_price * np.exp (- integ(int_rate, time_set))
#         upper_bdd = np.zeros(len(time_set))
#     return lower_bdd, upper_bdd


In [ ]:
def pricing_operator(t, bdd = 'Dirichlet'):
    time_loc = int((t - start_time)/dt)
    base_matrix = diags([A[time_loc],B[time_loc], C[time_loc]], [0,1,2], shape = (step_no, step_no+2))
    off_diag = diags([1], [1], shape = (step_no, step_no+2))
    if bdd == 'Dirichlet':
        matrix_left =  base_matrix + off_diag
        matrix_right = -base_matrix/2 + off_diag
    elif bdd == 'Neumann': 
        pass
    else:
        raise Exception('Unknown boundary condition')
    return matrix_left.tocsr(), matrix_right.tocsr()

In [ ]:
def option_price(asset, plot = False):
    outlayer = payoff(asset)[1:-1]
#     bdd_value = boundary(np.arange(0, end_time-start_time, dt))
    """the following is not fully optimised. 
    But as that is not related to linear system, we can ignore it (partially)"""
    for time in range(time_no+1):
        matrix_left, matrix_right = pricing_operator((time_no - time) * dt)[0], \
                    pricing_operator((time_no - time) * dt)[1]
        extra_vec =  # figure out what the heck this is
        
        outlayer = (matrix_right[:,1:-1] @ outlayer + bdd_value[0][time+1] * matrix_right[:,0].T \
                    + bdd_value[1][time+1] * matrix_right[:,-1].T + bdd_value[0][time] * matrix_left[:,0].T\
                    + bdd_value[1][time] * matrix_left[:,-1].T).A1
    return np.hstack(([0], outlayer))

In [ ]:
def BS_tester(asset, T=1, vol=0.2, r=0.05, D=0.01, K=100):
    moneyness = np.log(asset / K)
    d1 = (moneyness +(r-D + vol**2/2)*T)/(vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return asset * np.exp(-D*T)*norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2)

def plot(base_arr, *func):
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(111)
    plt.xlabel('asset value')
    plt.ylabel('option price')
    for f in func:
        ax.plot(base_arr, f(base_arr), '-o', label = f.__name__)
    ax.legend(loc ='best')
    plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 8))
asset = np.arange(50, 55, 1)
plot(asset, BS_tester, option_price)


In [ ]:
option_price(asset).shape, BS_tester(asset).shape

In [ ]:
add new staff